In [2]:
from scipy.linalg import eigh, eigvalsh
from scipy.sparse import coo_matrix
import numpy as np
import networkx,math,sys
import time
from scipy.stats import unitary_group, ortho_group
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

def psdrF(mat,r,d=None):
    if d is None:
        d = len(mat)
    vas,ves = eigh(mat)
    vas[vas<0]=0
    pvas=np.diag(vas*np.array([0]*(d-r)+[1]*r))
    ves = np.matrix(ves)
    return np.matmul(np.matmul(ves,pvas),ves.H)

def normF(mat,adj,d=None):
    if d is None:
        d = len(mat)
    return np.multiply(mat,adj) + 1*np.identity(d)

def ranH(d):
    Mr = np.random.rand(d,d)
    Mi = np.random.rand(d,d)
    M = (Mr+Mr.T)/2 + 1j*(Mi-Mi.T)/2
    return np.matrix(M)

def dimQ(ghf,r,t): # k is the rank of the projectors; r is the rank of the matrix, i.e., the dimension of the vectors; t is the number of steps in the see-saw method in each monitored period
    if isinstance(ghf,str):gh = networkx.from_graph6_bytes(ghf.encode())
    elif isinstance(ghf,list) or isinstance(ghf,np.matrix):gh = networkx.Graph(ghf)
    else:return('Accepted types:str of graph6 format,list of edges, np.matrix of adjacency matrix')
    d = len(gh)
    adjc = 1-np.matrix(networkx.adjacency_matrix(gh).A)-np.identity(d)
    iterM = ranH(d)
    tmp0 = d
    tmp1 = 2*d
    start0 = time.time()
    start = start0
    while abs(tmp1-tmp0)>1e-9 and start-start0<30:
        newtime = time.time()
        if newtime-start>3:
            iterM = iterM + ranH(d)/100
            start = newtime
        for i in range(t): # t is the number of steps in the see-saw method in each monitored period
            iterM = psdrF(iterM,r,d)
            iterM = normF(iterM,adjc,d)
        vas = eigvalsh(iterM,eigvals=(0,d-r-1))
        tmp0 = tmp1
        tmp1 = sum(abs(vas))
    #return [abs(tmp1-tmp0)<1e-9,tmp1<1e-9]
    if tmp1<1e-9:
        return 'True'
    else:
        return 'Undetermied'

def dimQM(ghf,r,lt): # lt is the number of rounds
    res = []
    for i in range(lt):
        tmp = dimQ(ghf,r,20)
        if tmp[0]:res.append(tmp[1])
        if tmp[1]<1e-6:return [len(res),tmp]
        if len(res)>=5:return [len(res),min(res)]
    return [len(res),min(res+[1000])]



In [3]:
from newdim import *

gh = [[0,1],[1,2],[2,3],[3,4],[0,4]] #edge format
gh = 'Dhc' #graph6 format
res = dimQ(gh,3,10) # graph, orthogonal rank to check, length of iteration in each small step (the larger the more accurate, then more time)
# The result 'Ture' means that we have a orthogonal representation with such a rank, 'Undetermined' means that it cannot be decided by this routine. We can always increase the targeted orthogonal rank to obtain the result 'True'.
print(res)

True


In [4]:
import csv
from csv import reader
from time import time

In [9]:
t0=time()
mini=1000

for vertices in range(37,48):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            if res != 'True':
                d=d+1
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta < mini:
                mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

37 	 0.861538461538462 	 16
38 	 0.774193548387097 	 6
39 	 0.869565217391304 	 24
40 	 0.888888888888889 	 8
41 	 0.738461538461539 	 2
42 	 0.880000000000000 	 42
43 	 0.821917808219178 	 212
44 	 0.857142857142857 	 4
45 	 0.846153846153846 	 4
46 	 0.835443037974684 	 2
47 	 0.779220779220779 	 92


In [44]:
t0=time()
mini=1000

for vertices in range(18,32):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            if res != 'True':
                d=d+1
            eta=2*lin[1]/(lin[1]+(vertices/d))
            mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

18 	 1 	 1.00000000000000 	 36
19 	 36/37 	 0.972972972972973 	 8
20 	 18/19 	 0.947368421052632 	 30
21 	 12/13 	 0.923076923076923 	 8
22 	 20/21 	 0.952380952380952 	 4
23 	 40/43 	 0.930232558139535 	 2
24 	 14/15 	 0.933333333333333 	 12
25 	 48/49 	 0.979591836734694 	 6
26 	 1 	 1.00000000000000 	 1
27 	 16/17 	 0.941176470588235 	 12
28 	 12/13 	 0.923076923076923 	 18
29 	 48/53 	 0.905660377358491 	 4
30 	 8/9 	 0.888888888888889 	 10
31 	 27/29 	 0.931034482758621 	 2


In [18]:
t0=time()
mini=1000

for vertices in range(33,48):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            #print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta <= mini:
                mini=min(mini,eta)
                print(lin,' ',1+int(vertices/lin[2]),' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
['hsb~vrwtDSrYfJhtwttn@rwVy_Fy?NQoJowAKktyYX^ieVsIrVpLT]uHT^fHX}PKSvkPc`FJga@Lja?bkdCkHmLCMPSJkYxDQmRjeaVQV}QbVBVyJDW]bxIP[Na{OXpDL\\rCuPPtu[', 4, 9.6999755, 4, 0.5614035087719298]   5   21
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
41 	 1.00000000000000 	 2


In [20]:
t0=time()
mini=1000

for vertices in range(18,31):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

18 	 1 	 1.00000000000000 	 36
19 	 1 	 1.00000000000000 	 8
20 	 1 	 1.00000000000000 	 30
21 	 1 	 1.00000000000000 	 8
22 	 1 	 1.00000000000000 	 4
23 	 1 	 1.00000000000000 	 2
24 	 1 	 1.00000000000000 	 12
25 	 1 	 1.00000000000000 	 6
26 	 1 	 1.00000000000000 	 1
27 	 1 	 1.00000000000000 	 12
28 	 1 	 1.00000000000000 	 18
29 	 1 	 1.00000000000000 	 4
30 	 1 	 1.00000000000000 	 10


In [5]:
t0=time()
mini=1000

for vertices in range(31,32):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

31 	 1 	 1.00000000000000 	 2


In [ ]:
t0=time()
mini=1000

for vertices in range(32,33):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()